In [10]:
import os
import time
import re
from tqdm import tqdm
from collections import Counter

In [3]:
dir_in = 'plain_texts_json_cut'

In [40]:
for f in tqdm(os.listdir(dir_in)):
    try:
        newname = '_'.join(re.findall('\w+', f.replace('_txt', '')))
        os.rename(os.path.join(dir_in, f), os.path.join(dir_in, newname))
    except:
        print(f, newname)
        continue

100%|██████████| 38419/38419 [00:00<00:00, 54410.07it/s]


In [50]:
with open('wv.txt', 'w') as f1:
    for f in tqdm(sorted(os.listdir(dir_in))):
        with open(os.path.join(dir_in, f)) as doc:
            tokens = [l.lower() for l in re.findall('\w+', doc.read().replace('\n', ' '))]
            doc_voc = Counter(tokens)
            s = ' '.join(f'{w}:{c}' for w, c in sorted(doc_voc.items()))
            f1.write(f'{f} {s}\n')
    
              

100%|██████████| 38412/38412 [04:53<00:00, 130.74it/s]


In [1]:
import artm

In [52]:
%%time
batch_vectorizer = artm.BatchVectorizer(data_path='wv.txt',
                                        data_format='vowpal_wabbit',
                                        target_folder='plain_texts_batches')

CPU times: user 7min 14s, sys: 5.35 s, total: 7min 19s
Wall time: 1min 20s


In [2]:
batch_vectorizer = artm.BatchVectorizer(data_path='plain_texts_batches', data_format='batches')

In [12]:
%%time
dictionary = artm.Dictionary()
dictionary.gather(data_path='plain_texts_batches')

CPU times: user 20.6 s, sys: 1.75 s, total: 22.4 s
Wall time: 18.5 s


In [13]:
dictionary.filter(min_tf=50, max_df_rate=0.9)

artm.Dictionary(name=ba278adf-3bff-4358-87c4-71a9a03eb76e, num_entries=78525)

In [54]:
dictionary.save(dictionary_path='plain_texts_batches/dictionary')

In [14]:
%%time
lda = artm.LDA(num_topics=200, alpha=0.01, beta=0.001, cache_theta=True,
               num_document_passes=5, dictionary=dictionary)

CPU times: user 1.48 s, sys: 158 ms, total: 1.64 s
Wall time: 1.33 s


In [15]:
%%time
lda.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=10)

CPU times: user 2h 24min 38s, sys: 59.5 s, total: 2h 25min 37s
Wall time: 19min 40s


In [16]:
lda.perplexity_value

[311.3059997558594,
 114.23505401611328,
 113.66322326660156,
 112.26839447021484,
 108.65650177001953,
 103.49784088134766,
 99.0697250366211,
 96.063232421875,
 94.06279754638672,
 92.64376831054688]

In [17]:
top_tokens = lda.get_top_tokens(num_tokens=10)
for i, token_list in enumerate(top_tokens):
     print ('Topic #{0}: {1}'.format(i, token_list))

Topic #0: ['организм', 'болезнь', 'клетка', 'мозг', 'врач', 'больной', 'кровь', 'вода', 'лечение', 'ребенок']
Topic #1: ['николай', 'поэт', 'стих', 'никита', 'москва', 'любить', 'пушкин', 'дом', 'георгий', 'письмо']
Topic #2: ['ричард', 'графиня', 'граф', 'ребенок', 'отец', 'мать', 'женщина', 'ваш', 'молодой', 'няня']
Topic #3: ['ваш', 'эдуард', 'господин', 'император', 'отец', 'полиция', 'кардинал', 'карлос', 'король', 'служба']
Topic #4: ['т', 'г', 'е', 'страна', 'развитие', 'война', 'общество', 'д', 'являться', 'политический']
Topic #5: ['ветер', 'лишь', 'тело', 'экспедиция', 'корабль', 'путь', 'снег', 'снова', 'блейд', 'мысль']
Topic #6: ['ваш', 'дон', 'ну', 'дама', 'сеньор', 'деньги', 'женщина', 'любовь', 'дом', 'жена']
Topic #7: ['ваш', 'господин', 'судья', 'однако', 'филипп', 'произносить', 'слишком', 'лишь', 'затем', 'дверь']
Topic #8: ['театр', 'г', 'писатель', 'писать', 'н', 'автор', 'пьеса', 'актер', 'написать', 'художник']
Topic #9: ['машина', 'девушка', 'полицейский', 'пар

In [18]:
%%time
lda.fit_offline(batch_vectorizer=batch_vectorizer, num_collection_passes=10)

CPU times: user 2h 21min 25s, sys: 51.1 s, total: 2h 22min 16s
Wall time: 19min 9s


In [19]:
lda.perplexity_value

[311.3059997558594,
 114.23505401611328,
 113.66322326660156,
 112.26839447021484,
 108.65650177001953,
 103.49784088134766,
 99.0697250366211,
 96.063232421875,
 94.06279754638672,
 92.64376831054688,
 91.58329772949219,
 90.76495361328125,
 90.1194076538086,
 89.60069274902344,
 89.17676544189453,
 88.82455444335938,
 88.52762603759766,
 88.27393341064453,
 88.0543441772461,
 87.86203002929688]

In [23]:
top_tokens = lda.get_top_tokens(num_tokens=20)
for i, token_list in enumerate(top_tokens):
     print ('Topic #{0}: {1}\n'.format(i, ' '.join(token_list)))

Topic #0: организм болезнь мозг клетка тело врач кровь здоровье больной лечение заболевание вещество орган вода пища состояние процесс ребенок также система

Topic #1: николай никита москва стих пушкин поэт писать георгий любить петербург читать письмо русский молодой московский помнить дом александр белый владимир

Topic #2: граф ричард графиня лидия няня отец ваш молодой женщина ребенок мать дочь синьор жена адриан дом комната гость муж сын

Topic #3: испанский эдуард испания испанец ваш кардинал сеньор дон диего карлос легион де легионер рим священник святой хуан консул сан отец

Topic #4: страна общество война политический власть т государство г е развитие экономический мировой система являться социальный население отношение н д право

Topic #5: снег лед гора ветер экспедиция скала вершина ледяной склон путь океан снежный лишь палатка высота час блейд горный подниматься пещера

Topic #6: ваш дон ну любить молодой дама уж ах господин муж просить женщина жена деньги дом входить сердц

In [24]:
top_tokens = lda.get_top_tokens(num_tokens=20)
for i, token_list in enumerate(top_tokens):
     print ('Topic #{0}: {1}\n'.format(i, ' '.join(token_list)))

Topic #0: организм болезнь мозг клетка тело врач кровь здоровье больной лечение заболевание вещество орган вода пища состояние процесс ребенок также система

Topic #1: николай никита москва стих пушкин поэт писать георгий любить петербург читать письмо русский молодой московский помнить дом александр белый владимир

Topic #2: граф ричард графиня лидия няня отец ваш молодой женщина ребенок мать дочь синьор жена адриан дом комната гость муж сын

Topic #3: испанский эдуард испания испанец ваш кардинал сеньор дон диего карлос легион де легионер рим священник святой хуан консул сан отец

Topic #4: страна общество война политический власть т государство г е развитие экономический мировой система являться социальный население отношение н д право

Topic #5: снег лед гора ветер экспедиция скала вершина ледяной склон путь океан снежный лишь палатка высота час блейд горный подниматься пещера

Topic #6: ваш дон ну любить молодой дама уж ах господин муж просить женщина жена деньги дом входить сердц